# COGS 108 - Data Checkpoint

# Names

- Tyler Le
- Aditya Tomar
- William Lynch
- Michael Mao
- Natalie Quach

<a id='research_question'></a>
# Research Question

Is there a positive correlation between the cost of living and the impact of natural disasters in terms of injuries, casualties, and property damage per capita at the county level? Furthermore, in which state does the impact of natural disasters affect cost of living the most?

# Dataset(s)
Please note that all data file names correspond to their name in our "datasets" folder.

**1. Frequency of Disasters By State**
- Dataset Name: Billion-Dollar Disasters By Year (CPI-Adjusted) (natural_disaster_frequencies.csv)
- Link to the dataset: https://www.ncdc.noaa.gov/billions/state-freq-data.csv
- Number of observations: 2228 
- Number of features: 9. These features are mostly numerical.

This dataset contains the count of natural disasters that cost more than 1 billion dollars for each state from 1980 - 2021. Each observation contains one state and the count of each natural disaster type for a given year.

**2. Types of Disaster By State/County**
- Dataset Name: Disaster Declarations Summaries (DisasterDeclarationsSummaries.csv)
- Link to the dataset: https://www.fema.gov/api/open/v1/DisasterDeclarationsSummaries.csv
- Number of observations: 62771
- Number of features: 23. Contains a combination of numerical and string data types.

This dataset contains all federally declared natural disasters from 1953-2022 (by year, state, county, and type) along with declared recovery programs. Each observation contains the type of natural disaster, when it occurred, and the state/county it occured in.


**3. National Risk Index (NRI)**
- Dataset Name: National Risk Index per County (NRI_Table_Counties.csv)
- Link to the dataset: https://hazards.fema.gov/nri/data-resources#csvDownload
- Number of observations: 3142
- Number of features: 365. Contains a combination of numerical and string data types.

Dataset from FEMA that identifies counties and states most at risk to 18 natural hazards. Includes data about expected annual losses from natural hazards, social vulnerability and community resilience. 

**4. States with Coastline**
- Dataset Name: States with Coastline (states_with_coastline.csv)
- Link to the dataset: https://worldpopulationreview.com/state-rankings/coastal-states
- Number of observations: 50
- Number of features: 2. Contains string data types.

This dataset contains whether or not each state in the United States has a coast. Each observation contains a state and its associated coast.


**5. Cost of Living**
- Dataset Name: Cost of Living (cost_of_living.csv)
- Link to the dataset: https://advisorsmith.com/wp-content/uploads/2021/02/advisorsmith_cost_of_living_index.csv
- Number of observations: 510
- Number of features: 3. Contains numerical and string data types.

Each observation in this dataset contains a state, the city associated with the state, and the Cost of Living Index. The Cost of Living Index measures the costs such as food and energy.

**6. Climate**
- Average Climate by County (Average_Climate_By_County.csv)
- https://www.ncdc.noaa.gov/cag/county/mapping
- Number of observations: 3137
- Number of features: 3. Contains a combination of numerical and string types.

This dataset contains the mean climate over a 5-year span from 2017 to 2022 for all counties in the USA except those in Hawaii. The climate is measured in Fahrenheit. This dataset is for comparing the correlation between natural disasters vs cost of living with climate vs cost of living because climate is a potential confounding variable that affects cost of living.



# Setup

In [83]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

# Data Cleaning

**Table #1 (Frequency of Disaster By State):** This dataset was fairly clean in that there were no missing values and each observation contains whether or not a state had a natural disaster event in a certain year. We decided to remove spaces from the column names and to replace them with underscores. Since each column describing a natural disaster contains the count of how many natural disasters of that type, there was little data cleaning needed for this dataset.

**Table #2 (Types of Disasters By State/County):** This dataset was fairly clean. We first focused on extracting the relevant columns, which were "state", "declarationType", "incidentType", "declarationTitle", and "declarationArea". These were the relevant variables since we want each state, the type of natural disaster, and whether it occured on a county level or not. We filtered the dataset to only contain natural disasters that occurred at the county level and standardized the column. We decided to keep the year it happened rather than the exact month and day since in our EDA in the future we would like to explore the natural disaster frequencies by decade. To make future analyses more convienient, we renamed some of the column names. Also, we checked for missing values and found that there were none. 

**Table #3 (National Risk Index):** This dataset was fairly clean. We focused on extracting the relevant columns, such as county, population size, National Risk Index score, and expected annual loss. These variables are important because we would like to compare counties per capita. We also decided to lowercase all the columns and replace spaces with underscores for consistency across all dataframes.

**Table #4 (States with Coastline):** This dataset was fairly clean. Originally, each observation in this dataset contained a state and its associated coast. If the state did not have a coast associated, it had a value "None". To aid with future analyses, such as fitting multiple linear regression models later on, we changed the "coast" column to be binary where 0 means that a state does not have a coast associated with it and 1 means that a state does have a coast associated with it.

**Table #6 (Average Climate by County)**
This dataset was very clean. All that we needed to change was remove ID numbers after the state abbreviations and change the column names from "Location ID" to "State", "Location" to "County", and "Value" to "Temperature (F)". 

We plan to merge each of the datasets by either the county or state columns. Additionally, we have datasets containing the number of casualties, number of injuries, and property damage amount per state for the years 2015-2020 that we plan on processing and cleaning after the checkpoint (these datasets are in the "datasets/nat_disast_bystate_deaths_cost” folder).

## Clean Table #1 (Frequency of Disaster By State)

In [ ]:
freq_df = pd.read_csv('datasets/natural_disaster_frequencies.csv')

# replace space with underscores in column names
freq_df.columns = freq_df.columns.str.replace(' ', '_')

# check for NaNs
assert(freq_df.isna().sum().sum() == 0)

freq_df

## Clean Table #2 (Types of Disaster By State/County)

In [ ]:
def standardize_county(str_in):
    try:
        if '(County)' in str_in:
            output = str_in.replace('(County)','')
        else:
            output = None
    except: 
        output = None

    return output


def standardize_year(str_in):
    try:
        output = str_in.split('T')[0]
        output = pd.to_datetime(str_in).year
    except:
        output = None
        
    return output

In [ ]:
disaster_type_df = pd.read_csv('datasets/DisasterDeclarationsSummaries.csv')

# select a subset of the columns
wanted_columns = ['state', 'declarationDate','incidentType','declarationTitle','designatedArea']

# rename the columns
disaster_type_df = disaster_type_df[wanted_columns].rename(columns={"declarationDate":"year", "designatedArea": "county", "incidentType":"disaster_type", "declarationTitle":"disaster_declaration"})

# Set "Statewide" to None and strip "(County)" from all counties
disaster_type_df['county'] = disaster_type_df['county'].apply(standardize_county)

# filter dataset to only include non-null 
disaster_type_df = disaster_type_df[~disaster_type_df['county'].isnull()]

# strip year column to only include year
disaster_type_df['year'] = disaster_type_df['year'].apply(standardize_year)

# sort by year
disaster_type_df = disaster_type_df.sort_values('year').reset_index(drop = True)

# check for no NaNs
assert(disaster_type_df.isna().sum().sum() == 0)

disaster_type_df

## Clean Table #3 (NRI)

In [ ]:
# Look at the "NRI Data Dictionary in the datasets/NRI_Table_Counties" to see what the cols mean
# EAL = "Expected Annual Lost", quantifies the anticipated economic damage resulting from natural hazards each year. 
# 1-100 scale

df_nri = pd.read_csv('datasets/NRI_Table_Counties/NRI_Table_Counties.csv')

# select a subset of the columns
wanted_cols = ['STATE','STATEABBRV','COUNTY','POPULATION','AREA','RISK_SCORE','RISK_RATNG','EAL_SCORE','EAL_RATNG']
df_nri = df_nri[wanted_cols]

# lowercase all columns
df_nri.columns = df_nri.columns.str.lower()

# rename columns
df_nri = df_nri.rename(columns={"stateabbrv":"state_abbrv", "risk_ratng":"risk_rating" ,"eal_ratng":"eal_rating"})

# make sure there are no NaNs
assert(df_nri.isna().sum().sum() == 0)

df_nri

## Clean Table #4 (States with Coastline)

In [ ]:
coastline_df = pd.read_csv('datasets/states_with_coastline.csv')
coastline_df

In [ ]:
# check data types for each column
coastline_df.dtypes

In [ ]:
coastline_df['coast'].value_counts()

In [ ]:
# check for missing values
coastline_df.isna().sum()

In [ ]:
# Binarize "coast" column. 0 = no coastline, 1 = yes coastline
def clean_coast(coast_val):
    coast_val = coast_val.lower()

    if "none" in coast_val:
        coast_val = coast_val.replace("none", "0")
        output = int(coast_val)
    else:
        output = 1
    
    return output

In [ ]:
# test function from above
assert clean_coast('None') == 0
assert clean_coast('Atlantic Ocean') == 1

In [ ]:
coastline_df['coast'] = coastline_df['coast'].apply(clean_coast)

In [ ]:
coastline_df

## Clean Table #5

In [84]:
df = pd.read_csv('datasets/cost_of_living.csv')
state_name = ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 
              'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 
              'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 
              'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 
              'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 
              'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 
              'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
state_abv = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA'
             , 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 
             'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

state_dict = dict(zip(state_abv,state_name))
df = df.replace({'State':state_dict})
geolocator = Nominatim(user_agent='find-county')
counties = list(zip(df.City,df.State))
counties
new_counties = []
for county in counties:
    city_state = (county[0]+' '+county[1])
    county = geolocator.geocode(city_state)[0].split(', ')[1]
    new_counties.append(county)
    # print(county,len(new_counties))


In [86]:
df['County'] = new_counties
county = df.pop('County')
df.insert(1,'County',county)
df.County.str.count("County").sum()

444

In [87]:
df

,City,County,State,Cost of Living Index
0,Abilene,Taylor County,TEXAS,89.1
1,Adrian,Lenawee County,MICHIGAN,90.5
2,Akron,Summit County,OHIO,89.4
3,Alamogordo,Otero County,NEW MEXICO,85.8
4,Albany,Dougherty County,GEORGIA,87.3
...,...,...,...,...
505,Wheeling,1528,WEST VIRGINIA,84.1
506,New London,New London County,CONNECTICUT,105.9
507,Daphne,Baldwin County,ALABAMA,96.6
508,Victoria,Texas,TEXAS,89.5


## Clean Table #6

In [ ]:
#Dataset for the mean climate for each county in the USA except those in Hawaii. The climate is measured over a 5-year span from 2017-2022. 
climate_df = pd.read_csv('datasets/Average_Climate_By_County.csv')
climate_df['Location ID'] = climate_df['Location ID'].apply([lambda x: x[0:2]])
climate_df = climate_df.rename(columns={'Location ID': 'State', 'Location': 'County', 'Value': 'Temperature (F)'})
climate_df

,State,County,Temperature (F)
0,AL,Autauga County,65.7
1,AL,Baldwin County,68.4
2,AL,Barbour County,66.0
3,AL,Bibb County,63.9
4,AL,Blount County,62.7
...,...,...,...
3132,AK,Skagway Municipality,30.9
3133,AK,Southeast Fairbanks Census Area,25.7
3134,AK,Wrangell City and Borough,40.0
3135,AK,Yakutat City and Borough,33.9
